In [49]:
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn.cluster import KMeans
retail_data = pd.read_csv('OnlineRetail.csv', encoding= 'unicode_escape')
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [50]:
##Convert 'InvoiceDate' to of type datetime
retail_data['InvoiceDate'] = pd.to_datetime(retail_data['InvoiceDate'], errors = 'coerce')

##Extract year and month information from 'InvoiceDate'
retail_data['yyyymm']=retail_data['InvoiceDate'].dt.strftime('%Y%m')

##Calculate revenue generated per order
retail_data['revenue'] = retail_data['UnitPrice'] * retail_data['Quantity']

## Calculate monthly revenue by aggregating the revenue on year month column
revenue_df = retail_data.groupby(['yyyymm'])['revenue'].sum().reset_index()
revenue_df.head()


,yyyymm,revenue
0,201012,748957.020
1,201101,560000.260
2,201102,498062.650
3,201103,683267.080
4,201104,493207.121


In [51]:
## Plot bar graph from revenue data frame with yyyymm column on x-axis and revenue on the y-axis.
import plotly.express as px

##Sort rows on year-month column
revenue_df.sort_values( by=['yyyymm'], inplace=True)

## plot a bar graph with year-month on x-axis and revenue on y-axis, update x-axis is of type category.
fig = px.bar(revenue_df, x="yyyymm", y="revenue", title="Monthly Revenue")
fig.update_xaxes(type='category')
fig.show()

In [52]:
## Calculate monthly active customers by aggregating the unique customer count on year-month column
active_customer_df = retail_data.groupby(['yyyymm'])['CustomerID'].nunique().reset_index()
active_customer_df.columns = ['yyyymm', 'No of Active customers']
active_customer_df.head()

,yyyymm,No of Active customers
0,201012,948
1,201101,783
2,201102,798
3,201103,1020
4,201104,899


In [53]:

## Plot bar graph from revenue data frame with yyyymm column on x-axis and No. of active customers on the y-axis.
fig = px.bar(active_customer_df, x="yyyymm", y="No of Active customers", title="Monthly Active customers")
fig.update_xaxes(type='category')
fig.show()

For this exercise, we will segment customers into the following groups:

- Low LTV: Less active or low revenue customers
- Mid-LTV: Fairly active and moderate revenue customers
- High LTV: High revenue customers – the segment that we don't want to lose

In [54]:
uk_data = retail_data.query("Country=='United Kingdom'").reset_index(drop=True)
uk_data.shape

(495478, 10)

In [55]:
uk_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,yyyymm,revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,201012,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,201012,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,20.34


In [56]:
from datetime import datetime, timedelta,date
t1 = pd.Timestamp("2011-06-01 00:00:00.054000")
t2 = pd.Timestamp("2011-03-01 00:00:00.054000")
t3 = pd.Timestamp("2011-12-01 00:00:00.054000")
uk_data_3m = uk_data[(uk_data.InvoiceDate < t1) & (uk_data.InvoiceDate >= t2)].reset_index(drop=True)
uk_data_6m = uk_data[(uk_data.InvoiceDate >= t1) & (uk_data.InvoiceDate < t3)].reset_index(drop=True)

In [57]:
uk_data_3m.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,yyyymm,revenue
0,545220,21955,DOORMAT UNION JACK GUNS AND ROSES,2,2011-03-01 08:30:00,7.95,14620.0,United Kingdom,201103,15.90
1,545220,48194,DOORMAT HEARTS,2,2011-03-01 08:30:00,7.95,14620.0,United Kingdom,201103,15.90
2,545220,22556,PLASTERS IN TIN CIRCUS PARADE,12,2011-03-01 08:30:00,1.65,14620.0,United Kingdom,201103,19.80
3,545220,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,2011-03-01 08:30:00,4.95,14620.0,United Kingdom,201103,14.85
4,545220,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,2011-03-01 08:30:00,3.75,14620.0,United Kingdom,201103,15.00


In [58]:
uk_data_3m['revenue'] = uk_data_3m['UnitPrice'] * uk_data_3m['Quantity']
max_date = uk_data_3m['InvoiceDate'].max() + timedelta(days=1)
rfm_data = uk_data_3m.groupby(['CustomerID']).agg({
        'InvoiceDate': lambda x: (max_date - x.max()).days,
        'InvoiceNo': 'count',
        'revenue': 'sum'})

In [59]:
rfm_data.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'revenue': 'MonetaryValue'}, inplace=True)

In [60]:
rfm_data

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747.0,7,35,1082.09
12748.0,1,582,4336.73
12749.0,8,54,782.10
12821.0,23,6,92.72
12823.0,63,1,459.00
...,...,...,...
18272.0,21,59,966.74
18273.0,66,1,51.00
18280.0,86,10,180.60


Here, we have calculated the R, F, and M values for the customers. Next, we need to divide customers into the R, F, and M groups. This grouping defines where a customer stands concerning the other customers in terms of the R, F, and M metrics. To calculate the R, F, and M groups, we will divide the customers into equal-sized groups based on their R, F, and M values, respectively. These were calculated in the previous code block.

In [61]:
r_grp = pd.qcut(rfm_data['Recency'], q=4, labels=range(3,-1,-1))
f_grp = pd.qcut(rfm_data['Frequency'], q=4, labels=range(0,4))
m_grp = pd.qcut(rfm_data['MonetaryValue'], q=4, labels=range(0,4))
rfm_data = rfm_data.assign(R=r_grp.values).assign(F=f_grp.values).assign(M=m_grp.values)
rfm_data['R'] = rfm_data['R'].astype(int)
rfm_data['F'] = rfm_data['F'].astype(int)
rfm_data['M'] = rfm_data['M'].astype(int)
rfm_data['RFMScore'] = rfm_data['R'] + rfm_data['F'] + rfm_data['M']
rfm_data.groupby('RFMScore')['Recency','Frequency','MonetaryValue'].mean()

<ipython-input-61-6e62db9efc9b>:9: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Recency,Frequency,MonetaryValue
RFMScore,,,
0,76.633588,5.206107,94.023359
1,58.500000,9.074074,158.990864
2,54.380208,13.968750,239.346354
3,38.091633,16.605578,271.437331
4,36.647619,24.180952,363.867000
5,34.000000,33.558252,535.423204
6,28.516432,44.920188,742.487324
7,18.431138,51.317365,974.310120
8,14.316547,80.064748,2192.578568


In [62]:
rfm_data['Segment'] = 'Low-Value'
rfm_data.loc[rfm_data['RFMScore']>4,'Segment'] = 'Mid-Value'
rfm_data.loc[rfm_data['RFMScore']>6,'Segment'] = 'High-Value'
rfm_data = rfm_data.reset_index()


In [63]:
uk_data_6m['revenue'] = uk_data_6m['UnitPrice'] * uk_data_6m['Quantity']
revenue_6m = uk_data_6m.groupby(['CustomerID']).agg({
        'revenue': 'sum'})
revenue_6m.rename(columns={'revenue': 'Revenue_6m'}, inplace=True)

In [64]:
revenue_6m = revenue_6m.reset_index()

In [65]:
merged_data = pd.merge(rfm_data, revenue_6m, how="left")
merged_data.fillna(0)

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Segment,Revenue_6m
0,12747.0,7,35,1082.09,3,2,3,8,High-Value,1666.11
1,12748.0,1,582,4336.73,3,3,3,9,High-Value,18679.01
2,12749.0,8,54,782.10,3,3,3,9,High-Value,2323.04
3,12821.0,23,6,92.72,2,0,0,2,Low-Value,0.00
4,12823.0,63,1,459.00,0,0,2,2,Low-Value,765.00
...,...,...,...,...,...,...,...,...,...,...
1835,18272.0,21,59,966.74,2,3,3,8,High-Value,1730.16
1836,18273.0,66,1,51.00,0,0,0,0,Low-Value,102.00
1837,18280.0,86,10,180.60,0,0,0,0,Low-Value,0.00
1838,18283.0,9,100,217.15,3,3,1,7,High-Value,1351.83


In the flowing code block, we are using the revenue_6m columns, which is the lifetime value of a customer, and creating three groups called Low LTV, Mid LTV, and High LTV using K-means clustering. Again, you can verify the optimal number of clusters using the elbow method mentioned previously:

In [66]:
merged_data = merged_data[merged_data['Revenue_6m']<merged_data['Revenue_6m'].quantile(0.99)]

In [67]:
merged_data.head()

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Segment,Revenue_6m
0,12747.0,7,35,1082.09,3,2,3,8,High-Value,1666.11
1,12748.0,1,582,4336.73,3,3,3,9,High-Value,18679.01
2,12749.0,8,54,782.10,3,3,3,9,High-Value,2323.04
4,12823.0,63,1,459.00,0,0,2,2,Low-Value,765.00
7,12836.0,28,62,814.71,1,3,3,7,High-Value,951.46


In [69]:
kmeans=KMeans(n_clusters=3)
kmeans.fit(merged_data[['Revenue_6m']])
merged_data['LTVCluster']=kmeans.predict(merged_data[['Revenue_6m']])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [70]:
merged_data.groupby('LTVCluster')['Revenue_6m'].describe()

,count,mean,std,min,25%,50%,75%,max
LTVCluster,,,,,,,,
0,186.0,4137.019892,1477.396388,2503.30,2964.2275,3635.200,4987.535,8910.04
1,21.0,14123.309048,3653.311262,9313.18,11464.6700,12913.990,16756.310,20530.00
2,1170.0,828.670189,621.405282,-609.40,324.4675,663.525,1245.465,2447.57


In [71]:
feature_data = pd.get_dummies(merged_data)
feature_data.reset_index()
feature_data.head(5)

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Revenue_6m,LTVCluster,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value
0,12747.0,7,35,1082.09,3,2,3,8,1666.11,2,1,0,0
1,12748.0,1,582,4336.73,3,3,3,9,18679.01,1,1,0,0
2,12749.0,8,54,782.10,3,3,3,9,2323.04,2,1,0,0
4,12823.0,63,1,459.00,0,0,2,2,765.00,2,0,1,0
7,12836.0,28,62,814.71,1,3,3,7,951.46,2,1,0,0


In [72]:
from sklearn.metrics import classification_report,confusion_matrix
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

# feature_data = pd.get_dummies(merged_data)
X = feature_data.drop(['LTVCluster', 'Revenue_6m'], axis=1)
y = feature_data['LTVCluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [73]:
xgb_classifier = xgb.XGBClassifier(max_depth=5, objective='multi:softprob')
xgb_model = xgb_classifier.fit(X_train, y_train)
acc = xgb_model.score(X_test,y_test)
print(acc)

0.855072463768116


In [74]:
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.21      0.27      0.24        11
           1       0.00      0.00      0.00         2
           2       0.94      0.92      0.93       125

    accuracy                           0.86       138
   macro avg       0.39      0.40      0.39       138
weighted avg       0.87      0.86      0.86       138

